# Data curation

### Import some necessary functions

In [ ]:
import os, pydicom, sys
import numpy as np
from Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image, plot_scroll_Image
import SimpleITK as sitk
from Image_Array_And_Mask_From_Dicom_RT import Dicom_to_Imagestack

## Finding the Data

### Find where we put our data

In [ ]:
data_path = r'K:\Morfeus\BMAnderson\CNN\Data\Data_Liver\Liver_Disease_Ablation_Segmentation\Images'
print('We have ' + str(len(os.listdir(data_path))) + ' patients!')

## Ensuring contour fidelity...

### Note that we've set 'get_images_mask' to False, this means we won't be getting any of the image data, just looking at the dicom RT files

In [ ]:
Dicom_Reader = Dicom_to_Imagestack(get_images_mask=False)

In [ ]:
Dicom_Reader.down_folder(data_path)

### What ROI names do we have?

#### This will tell us all the unique roi names, hence all_rois

In [ ]:
for roi in Dicom_Reader.all_rois:
    print(roi)

## Make contour associations

#### We have quite a few contour names here.. now, we can either change the ROI names in the RT files, or make an associations file

#### The associations file associates a contour name with another one {'Current contour':'Desired name'}

In [ ]:
associations = {'Liver_BMA_Program_4':'Liver',
                'bma_liver':'Liver',
                'best_liver':'Liver',
                'tried_liver':'Liver'}

### Tell the Dicom_Reader that we want to set the associations, get the images and mask for contour 'Liver'

In [ ]:
Dicom_Reader.set_associations(associations)
Dicom_Reader.set_get_images_and_mask(True)
Dicom_Reader.set_contour_names(['liver'])

### Re-write RTs
#### This is commented out, because if I run it, then the example above won't show any different contour names

In [ ]:
# Dicom_Reader.associations = associations
# for RT in Dicom_Reader.all_RTs:
#     Dicom_Reader.rewrite_RT(RT)

## Pulling images and mask

### We'll first do this with one patient

In [ ]:
patient_data_path = os.path.join(data_path,os.listdir(data_path)[0],'CT 2')
Dicom_Reader.Make_Contour_From_directory(patient_data_path)
print('Done!')

## View images

In [ ]:
%matplotlib inline

### The images and mask are saved within the Dicom_Reader class, so we just have to load them

In [ ]:
Images = Dicom_Reader.ArrayDicom
mask = Dicom_Reader.mask # This is the mask

### Viewing nifti handles

In [ ]:
dicom_handle = Dicom_Reader.dicom_handle
annotations_handle = Dicom_Reader.annotation_handle

### Threshold

In [ ]:
Images[Images<-200] = -200
Images[Images>200] = 200

In [ ]:
plot_Image_Scroll_Bar_Image(Images)

In [ ]:
Images[mask==1] += 300

# Turn predictions into an RT

## Your prediction needs to be of the form [# Images, rows, columns, #classes + 1], the [...,0] is background

In [ ]:
prediction = np.zeros(Images.shape+(2,)) # Two classes, background and test

In [ ]:
prediction[...,300:400,300:400,1] = 1

### Need to pass the prediction, an output directory, and ROI_Names list equal to the number of classes

In [ ]:
help(Dicom_Reader.with_annotations)

In [ ]:
out_path = os.path.join(data_path,os.listdir(data_path)[0],'CT 2','new_RT')
ROI_Names=['test']

In [ ]:
Dicom_Reader.with_annotations(prediction,out_path=out_path,ROI_Names=ROI_Names)

# Recap

### Checking ROI contour names and making associations

### Loading in image and mask from desired contour name

### Viewing images and mask

### Turning predictions into an RT Structure

# Separate into Train/Test/Validation

### This is also important, but I would recommend using the 'Parallel' approach available in https://github.com/brianmanderson/Dicom_Data_to_Numpy_Arrays